In [1]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import os
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

In [3]:
train_npz = np.load('/kaggle/input/argoverse-modified/train.npz')
train_data = train_npz['data']
test_npz  = np.load('/kaggle/input/argoverse-modified/test_input.npz')
test_data  = test_npz['data']

In [4]:
print(train_data.shape, test_data.shape)

# Split once for later use
X_train = train_data[..., :50, :]
Y_train = train_data[:, 0, 50:, :2]

(10000, 50, 110, 6) (2100, 50, 50, 6)


In [5]:
def rotate(x, y, heading):
    cos_theta = np.cos(-heading)
    sin_theta = np.sin(-heading)
    x_rot = x * cos_theta - y * sin_theta
    y_rot = x * sin_theta + y * cos_theta
    return x_rot, y_rot

In [6]:
class MotionEmbeddingMLP(nn.Module):
    def __init__(self, input_dim = 12, hidden_dim=32, output_dim=16):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        # x: (num_agents, seq_len, input_dim)
        return self.mlp(x)

In [7]:
class TrajectoryDatasetTrain(Dataset):
    def __init__(self, data, scale=10.0, augment=True):
        """
        data: Shape (N, 50, 110, 6) Training data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        augment: Whether to apply data augmentation (only for training)
        """
        self.data = data
        self.scale = scale
        self.augment = augment
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx]
        # Getting 50 historical timestamps and 60 future timestamps
        hist = scene[:, :50, :].copy()    # (agents=50, time_seq=50, 6)
        future = torch.tensor(scene[0, 50:, :2].copy(), dtype=torch.float32)  # (60, 2)
        
        # Data augmentation(only for training)
        if self.augment:
            if np.random.rand() < 0.5:
                theta = np.random.uniform(-np.pi, np.pi)
                R = np.array([[np.cos(theta), -np.sin(theta)],
                              [np.sin(theta),  np.cos(theta)]], dtype=np.float32)
                # Rotate the historical trajectory and future trajectory
                hist[..., :2] = hist[..., :2] @ R
                hist[..., 2:4] = hist[..., 2:4] @ R
                future = future @ R
            if np.random.rand() < 0.5:
                hist[..., 0] *= -1
                hist[..., 2] *= -1
                future[:, 0] *= -1

            # Gaussian position jitter
            if np.random.rand() < 0.5:
                noise = np.random.normal(0, 0.1, size=hist[..., :2].shape)
                hist[..., :2] += noise

            # Time warping
            if np.random.rand() < 0.3:
                scale = np.random.uniform(0.9, 1.1)
                hist[..., :2] *= scale
                hist[..., 2:4] *= scale
                future *= scale

            # Velocity perturbation
            if np.random.rand() < 0.4:
                perturb = np.random.normal(0, 0.05, size=hist[..., 2:4].shape)
                hist[..., 2:4] += perturb
                hist[..., :2] += perturb * 0.1  # assuming 0.1s per timestep

        # Use the last timeframe of the historical trajectory as the origin
        origin = hist[0, 49, :2].copy()  # (2,)
        hist[..., :2] = hist[..., :2] - origin
        future = future - origin

        # Normalize the historical trajectory and future trajectory
        hist[..., :4] = hist[..., :4] / self.scale
        future = future / self.scale

        vx = hist[..., 2]
        vy = hist[..., 3]
        ego_heading = np.arctan2(vy[0], vx[0])
        ego_heading = ego_heading[None, :] 
        dvx = np.diff(vx, axis=1, prepend=vx[:, 0:1])
        dvy = np.diff(vy, axis=1, prepend=vy[:, 0:1])
        acceleration = np.sqrt(dvx**2 + dvy**2)

        acc_x_ego, acc_y_ego = rotate(dvx, dvy, ego_heading)

        vel_x_ego, vel_y_ego = rotate(vx, vy, ego_heading)

        theta = np.arctan2(vy, vx)
        dtheta = np.diff(theta, axis=1, prepend=theta[:, 0:1])

        ego_pos = hist[0:1, :, :2]  # (1, 50, 2)
        rel_pos = hist[:, :, :2] - ego_pos  # (50, 50, 2)
        dist_to_ego = np.linalg.norm(rel_pos, axis=-1, keepdims=True)  # (50, 50, 1)
        
        rel_x = rel_pos[:, :, 0]
        rel_y = rel_pos[:, :, 1]

        rot_x, rot_y = rotate(rel_x, rel_y, ego_heading)

        rotated_rel = np.stack([rot_x, rot_y], axis=-1)
        rotated_vel = np.stack([vel_x_ego, vel_y_ego], axis=-1) # (50, 50, 2)
        rotated_acc = np.stack([acc_x_ego, acc_y_ego], axis=-1)


        hist = np.concatenate([hist, dvx[..., None], dvy[..., None], acceleration[..., None], dtheta[..., None],
        rel_pos, rotated_rel, rotated_vel, rotated_acc, dist_to_ego], axis=-1)
        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            y=future.type(torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )

        return data_item
    

class TrajectoryDatasetTest(Dataset):
    def __init__(self, data, scale=10.0):
        """
        data: Shape (N, 50, 110, 6) Testing data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        """
        self.data = data
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Testing data only contains historical trajectory
        scene = self.data[idx]  # (50, 50, 6)
        hist = scene.copy()
        
        origin = hist[0, 49, :2].copy()
        hist[..., :2] = hist[..., :2] - origin
        hist[..., :4] = hist[..., :4] / self.scale

        vx = hist[..., 2]
        vy = hist[..., 3]
        ego_heading = np.arctan2(vy[0], vx[0])
        ego_heading = ego_heading[None, :] 
        dvx = np.diff(vx, axis=1, prepend=vx[:, 0:1])
        dvy = np.diff(vy, axis=1, prepend=vy[:, 0:1])
        acceleration = np.sqrt(dvx**2 + dvy**2)

        acc_x_ego, acc_y_ego = rotate(dvx, dvy, ego_heading)

        vel_x_ego, vel_y_ego = rotate(vx, vy, ego_heading)

        theta = np.arctan2(vy, vx)
        dtheta = np.diff(theta, axis=1, prepend=theta[:, 0:1])

        ego_pos = hist[0:1, :, :2]  # (1, 50, 2)
        rel_pos = hist[:, :, :2] - ego_pos
        dist_to_ego = np.linalg.norm(rel_pos, axis=-1, keepdims=True)  # (50, 50, 1)

        rel_x = rel_pos[:, :, 0]
        rel_y = rel_pos[:, :, 1]

        rot_x, rot_y = rotate(rel_x, rel_y, ego_heading)

        rotated_rel = np.stack([rot_x, rot_y], axis=-1)
        rotated_vel = np.stack([vel_x_ego, vel_y_ego], axis=-1) # (50, 50, 2)
        rotated_acc = np.stack([acc_x_ego, acc_y_ego], axis=-1)

        hist = np.concatenate([hist, dvx[..., None], dvy[..., None], acceleration[..., None], dtheta[..., None],
        rel_pos, rotated_rel, rotated_vel, rotated_acc, dist_to_ego], axis=-1)

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )
        return data_item

In [8]:
torch.manual_seed(251)
np.random.seed(42)

scale = 7.0

N = len(train_data)
val_size = int(0.1 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using CUDA GPU


In [9]:
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 50):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # (d_model/2)

        pe[:, 0::2] = torch.sin(position * div_term)  # even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # odd indices

        self.register_buffer('pe', pe)  # makes sure it's not a model parameter

    def forward(self, seq_len):
        return self.pe[:seq_len]

In [10]:
class SpatialBlock(nn.Module):
    def __init__(self, hidden_size, nhead):
        super().__init__()
        self.attn = nn.MultiheadAttention(hidden_size, nhead, batch_first=True)

    def forward(self, ego, others):
        # ego: [B, 1, H], others: [B, N, H]
        attn_out, _ = self.attn(ego, others, others)
        return ego + attn_out  # [B, 1, H]

class AttentionPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.query = nn.Parameter(torch.randn(hidden_dim))  # learnable [H]
        self.scale = hidden_dim ** 0.5

    def forward(self, x, mask=None):
        # x: [B, N, T, H]
        B, N, T, H = x.shape
        q = self.query.view(1, 1, 1, H)  # [1, 1, 1, H]
        attn_scores = (x * q).sum(dim=-1) / self.scale  # [B, N, T]

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask.squeeze(-1) == 0, -1e9)

        attn_weights = F.softmax(attn_scores, dim=-1)  # [B, N, T]
        pooled = (attn_weights.unsqueeze(-1) * x).sum(dim=2)  # [B, N, H]
        return pooled

class AgentAwarePredictor(nn.Module):
    def __init__(self, input_stats=None, target_stats=None):
        super().__init__()
        self.hidden_size = 128
        self.nhead = 4
        self.num_layers = 2
        self.num_agent_types = 10
        self.dropout_rate = 0.0

        self.time_pool = AttentionPooling(self.hidden_size * 2)

        agent_embed_dim = self.hidden_size // 4
        self.agent_embed = nn.Embedding(self.num_agent_types, agent_embed_dim)

        self.input_proj = nn.Linear(19 + agent_embed_dim, self.hidden_size)
        self.input_norm = nn.LayerNorm(self.hidden_size)
        self.pos_encoder = PositionalEncoding(self.hidden_size)

        encoder_layer = TransformerEncoderLayer(
            d_model=self.hidden_size * 2,
            nhead=self.nhead,
            dim_feedforward=2 * self.hidden_size,
            dropout=self.dropout_rate,
            batch_first=True
        )
        self.temporal_encoder = TransformerEncoder(encoder_layer, self.num_layers)

        self.cross_attn = SpatialBlock(self.hidden_size * 2, self.nhead)

        self.decoder = nn.Sequential(
            nn.Linear(self.hidden_size * 2, 256),
            nn.ReLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(256, 60 * 2)
        )

    def forward(self, data):
        x = data.x  # shape: [B*50, 50, 19]
        x = x.view(-1, 50, 50, 19)  # [B, N=50, T=50, F=19]
        B, N, T, F = x.shape
    
        # Mask dead agents
        dead_agents = (x.abs().sum(dim=(2, 3)) == 0)  # [B, N]
        alive_mask = ~dead_agents  # [B, N]
        max_agents = alive_mask.sum(dim=1).max().item()
        new_x = torch.zeros(B, max_agents, T, F, device=x.device)
    
        for b in range(B):
            new_x[b, :alive_mask[b].sum()] = x[b, alive_mask[b]]
    
        x = new_x
        B, N, T, F = x.shape  # updated N
    
        # Agent types from feature index 5
        agent_types = x[:, :, 0, 5].long()
        type_embed = self.agent_embed(agent_types).unsqueeze(2).expand(-1, -1, T, -1)
    
        # Combine features + agent embedding
        x_combined = torch.cat([x, type_embed], dim=-1)  # [B, N, T, F+E]
        x_proj = self.input_proj(x_combined)
        x_proj = self.input_norm(x_proj)
    
        # Flatten for temporal encoding
        x_flat = x_proj.view(B * N, T, self.hidden_size)
    
        # Positional encoding
        pos_emb = self.pos_encoder(T).unsqueeze(0).expand(B * N, -1, -1)
        x_flat = torch.cat([x_flat, pos_emb], dim=-1)  # [B*N, T, H*2]
    
        # Temporal Transformer
        x_encoded = self.temporal_encoder(x_flat).view(B, N, T, -1)
    
        # Time attention pooling
        mask = (x.abs().sum(dim=-1) > 0).float().unsqueeze(-1)  # [B, N, T, 1]
        x_encoded = self.time_pool(x_encoded, mask=mask)  # [B, N, H]
    
        # Ego attends to all
        h_ego = x_encoded[:, 0:1, :]  # [B, 1, H]
        h_attended = self.cross_attn(h_ego, x_encoded)  # [B, 1, H]
        h_attended = h_attended.squeeze(1)  # [B, H]
    
        pred_norm = self.decoder(h_attended).view(B, 60, 2)
        return pred_norm

    def predict_denorm(self, x):
        return self.forward(x)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 50):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # (d_model/2)

        pe[:, 0::2] = torch.sin(position * div_term)  # even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # odd indices

        self.register_buffer('pe', pe)  # makes sure it's not a model parameter

    def forward(self, seq_len):
        return self.pe[:seq_len]

In [11]:
model = AgentAwarePredictor().to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) # You can try different schedulers
early_stopping_patience = 10
best_val_loss = float('inf')
no_improvement = 0
criterion = nn.MSELoss()

In [12]:
def combined_loss_fn(pred_norm, y_norm, batch, fde_weight=0.2):
    """
    Combined loss function with MSE + FDE in world coordinates
    """
    # 1) Raw MSE (in normalized coordinates)
    raw_loss = F.mse_loss(pred_norm, y_norm)
    
    # 2) Denormalize to world coordinates
    scale = batch.scale.view(-1, 1, 1)   # (B,1,1)
    origin = batch.origin.view(-1, 1, 2)  # (B,1,2)
    pred_world = pred_norm * scale + origin  # (B,60,2)
    y_world = y_norm * scale + origin
    
    # 3) World MSE
    world_loss = F.mse_loss(pred_world, y_world)
    
    # 4) Final Displacement Error (FDE) at t=60 in world coords
    fde = F.mse_loss(pred_world[:, -1, :], y_world[:, -1, :])
    
    # 5) Combined loss: world_loss + fde_weight * fde
    total_loss = world_loss + fde_weight * fde
    
    return total_loss, raw_loss, world_loss, fde

In [13]:
#torch.cuda.empty_cache()
for epoch in tqdm.tqdm(range(100), desc="Epoch", unit="epoch"):
    # ---- Training ----
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(batch.num_graphs, 60, 2)
        loss, raw_loss, world_loss, fde = combined_loss_fn(pred, y, batch)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        train_loss += loss.item()
    
    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(batch.num_graphs, 60, 2)
            
            combined_val_loss, _, _, _ = combined_loss_fn(pred, y, batch)
            val_loss += combined_val_loss.item()

            # show MAE and MSE with unnormalized data
            pred = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            y = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            val_mae += nn.L1Loss()(pred, y).item()
            val_mse += nn.MSELoss()(pred, y).item()
    
    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    scheduler.step()
    # scheduler.step(val_loss)
    print("epoch 1 almost done")
    
    tqdm.tqdm.write(f"Epoch {epoch:03d} | Learning rate {optimizer.param_groups[0]['lr']:.6f} | train normalized MSE {train_loss:8.4f} | val normalized MSE {val_loss:8.4f}, | val MAE {val_mae:8.4f} | val MSE {val_mse:8.4f}")
    if val_loss < best_val_loss - 1e-3:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        no_improvement += 1
        if no_improvement >= early_stopping_patience:
            no_improvement = 0
            optimizer.param_groups[0]['lr'] /= 2
            # print("Early stop!")
            # break

Epoch:   1%|          | 1/100 [00:53<1:28:43, 53.77s/epoch]

epoch 1 almost done
Epoch 000 | Learning rate 0.001000 | train normalized MSE  73.4450 | val normalized MSE  40.9533, | val MAE   2.9722 | val MSE  24.2212


Epoch:   2%|▏         | 2/100 [01:45<1:26:12, 52.78s/epoch]

epoch 1 almost done
Epoch 001 | Learning rate 0.001000 | train normalized MSE  33.8285 | val normalized MSE  30.0519, | val MAE   2.6652 | val MSE  17.5594


Epoch:   3%|▎         | 3/100 [02:37<1:24:34, 52.31s/epoch]

...

epoch 1 almost done
Epoch 097 | Learning rate 0.000000 | train normalized MSE  12.4232 | val normalized MSE  13.4529, | val MAE   1.2029 | val MSE   7.1586


Epoch:  99%|█████████▉| 99/100 [1:26:46<00:52, 52.82s/epoch]

epoch 1 almost done
Epoch 098 | Learning rate 0.000000 | train normalized MSE  12.3710 | val normalized MSE  13.4496, | val MAE   1.2027 | val MSE   7.1560


Epoch: 100%|██████████| 100/100 [1:27:39<00:00, 52.59s/epoch]

epoch 1 almost done
Epoch 099 | Learning rate 0.000000 | train normalized MSE  12.3581 | val normalized MSE  13.4495, | val MAE   1.2025 | val MSE   7.1560


In [14]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

best_model = torch.load("best_model.pt")
#model = LinearRegressionModel().to(device)
# model = MLP(50 * 50 * 6, 60 * 2).to(device)
model = AgentAwarePredictor().to(device)

model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        pred_norm = model(batch)
        
        # Reshape the prediction to (N, 60, 2)
        pred = pred_norm * batch.scale.view(-1,1,1) + batch.origin.unsqueeze(1)
        pred_list.append(pred.cpu().numpy())
pred_list = np.concatenate(pred_list, axis=0)  # (N,60,2)
pred_output = pred_list.reshape(-1, 2)  # (N*60, 2)
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('submission.csv', index=True, float_format='%.8f')